In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
# Set input and output folders
input_folder = "/kaggle/input/thai-handwritten-characters-recognition/train/train"
# output_folder = "path/to/output/folder"

In [3]:
# Load dataset and preprocess images
images = []
labels = []

In [4]:
# Loop through input folder and load images and labels
for label in os.listdir(input_folder):
    label_folder = os.path.join(input_folder, label)
    for file in os.listdir(label_folder):
        image = cv2.imread(os.path.join(label_folder, file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))  # Resize to a fixed size
        images.append(image)
        labels.append(label)  # Use folder name as label

In [5]:
# Convert data to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [6]:
# Extract SIFT features
sift = cv2.xfeatures2d.SIFT_create()
sift_features = []
for image in images:
    _, descriptors = sift.detectAndCompute(image, None)
    sift_features.append(descriptors)

In [8]:
sift_features = np.array(sift_features)

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sift_features, labels, test_size=0.2, random_state=42)

In [ ]:
len(X_train)

In [10]:
# Train SVM classifier
svm = SVC(kernel='rbf', C=1, probability=True, random_state=42)
svm.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# Predict labels for testing data
y_pred = svm.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

In [ ]:
# Define the function to compute GFoDRs
def compute_gfodrs(image):
    # Compute the gradient magnitude and orientation
    grad_mag, grad_orient = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False, feature_vector=False)
    
    # Split the image into 4 regions
    height, width = image.shape[:2]
    h_half, w_half = height // 2, width // 2
    regions = [(0, h_half, 0, w_half), (0, h_half, w_half, width), (h_half, height, 0, w_half), (h_half, height, w_half, width)]
    
    gfodrs = []
    for region in regions:
        # Extract the gradient magnitudes and orientations for the region
        region_mag = grad_mag[region[0]:region[1], region[2]:region[3]]
        region_orient = grad_orient[region[0]:region[1], region[2]:region[3]]
        
        # Compute the GFoDR for the region
        gfodr = np.zeros(9)
        for i in range(9):
            indices = np.where(np.logical_and(region_orient >= i * 20, region_orient < (i + 1) * 20))
            gfodr[i] = np.sum(region_mag[indices])
        gfodrs.append(gfodr)
    
    return np.concatenate(gfodrs)

# Define the function to compute HOGFoDRs
def compute_hogfodrs(image):
    # Compute the HOG features
    hog_feature = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(2, 2))
    
    # Compute the GFoDRs
    gfodr_feature = compute_gfodrs(image)
    
    # Concatenate the HOG and GFoDR features
    features = np.concatenate((hog_feature, gfodr_feature))
    
    return features